In [9]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest

trading_client = TradingClient('PKXYP4B1C236T3WB7260', '4FWCFUxloEaFcFdmzBR1HohGloWC1a3n47jgGjSc')

# Get our account information.
account = trading_client.get_account()

# Check if our account is restricted from trading.
if account.trading_blocked:
    print('Account is currently restricted from trading.')

# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))

$2000000 is available as buying power.


In [11]:
aapl_asset = trading_client.get_asset('AAPL')

if aapl_asset.tradable:
    print('We can trade AAPL.')

We can trade AAPL.


In [14]:
# Initialize Alpaca API
api_key = "PKXYP4B1C236T3WB7260"
api_secret = "4FWCFUxloEaFcFdmzBR1HohGloWC1a3n47jgGjSc"

In [52]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime, timedelta
import pandas as pd
from scipy.stats import zscore
import time

# Initialize Alpaca API
api_key = "PKXYP4B1C236T3WB7260"
api_secret = "4FWCFUxloEaFcFdmzBR1HohGloWC1a3n47jgGjSc"

trading_client = TradingClient(api_key, api_secret, paper=True)
historical_data_client = StockHistoricalDataClient(api_key, api_secret)

# Define pairs and beta
pairs = [
    {'asset1': 'AAPL', 'asset2': 'MSFT', 'beta': 0.75},
    # Add more pairs here
]

# Pairs trading logic
for pair in pairs:
    asset1 = pair['asset1']
    asset2 = pair['asset2']
    beta = pair['beta']

    # Fetch historical data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=120)  # Last 30 days

    bars_request = StockBarsRequest(
        symbol_or_symbols=[asset1, asset2],
        timeframe=TimeFrame.Day,
        start=start_date,
        end=end_date
    )

    bars = historical_data_client.get_stock_bars(bars_request)
    df1 = pd.DataFrame(bars[asset1])
    df2 = pd.DataFrame(bars[asset2])

    for col in df1.columns:
        # Extract column name from the first row
        column_name = df1[col].iloc[0][0] if isinstance(df1[col].iloc[0], tuple) else None

        # Update each row in the column
        df1[col] = df1[col].apply(lambda x: x[1] if isinstance(x, tuple) else x)

        # Rename the column if column_name is not None
        if column_name:
            df1.rename(columns={col: column_name}, inplace=True)

    # Repeat the same process for df2
    for col in df2.columns:
        # Extract column name from the first row
        column_name = df2[col].iloc[0][0] if isinstance(df2[col].iloc[0], tuple) else None

        # Update each row in the column
        df2[col] = df2[col].apply(lambda x: x[1] if isinstance(x, tuple) else x)

        # Rename the column if column_name is not None
        if column_name:
            df2.rename(columns={col: column_name}, inplace=True)
            
    # Assume both dataframes are aligned and have the same length
    df = pd.DataFrame({
        asset1: df1["close"],
        asset2: df2["close"]
    })

    # Calculate spread and Z-score
    df['spread'] = df[asset2] - beta * df[asset1]
    df['zscore'] = zscore(df['spread'])

    # Trading signals
    UL = 1  # Upper limit
    LL = -1  # Lower limit

    holding_position = False  # Flag to indicate if we are holding a position

    for i in range(len(df)):
        if not holding_position:
            if df['zscore'].iloc[i] > UL:
                # Short asset1 and Long asset2
                trading_client.submit_order(
                    order_data=MarketOrderRequest(
                        symbol=asset1,
                        qty=1 * beta,
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.DAY
                    )
                )
                trading_client.submit_order(
                    order_data=MarketOrderRequest(
                        symbol=asset2,
                        qty=1,
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.DAY
                    )
                )
                holding_position = True

            elif df['zscore'].iloc[i] < LL:
                # Long asset1 and Short asset2
                trading_client.submit_order(
                    order_data=MarketOrderRequest(
                        symbol=asset1,
                        qty=1 * beta,
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.DAY
                    )
                )
                trading_client.submit_order(
                    order_data=MarketOrderRequest(
                        symbol=asset2,
                        qty=1,
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.DAY
                    )
                )
                holding_position = True

        elif holding_position:
            if abs(df['zscore'].iloc[i]) < 0.5:
                # Close the existing position
                # (You'll need to add logic to close the position based on your last trade side)

                holding_position = False

        # Sleep to avoid hitting rate limits
        time.sleep(1)
        